In [2]:
from google.colab import drive
drive.mount('/content/Google_Drive')

Mounted at /content/Google_Drive


In [3]:
import cv2 as cv
import glob as gb
import matplotlib.pyplot as plt
import csv 

path=r'/content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Graph/graph.pb'
net= cv.dnn.readNetFromTensorflow(path)
path =gb.glob('/content/Google_Drive/MyDrive/College /Human Pose Estimation Using Open CV/Images/Input/*.jpg')

In [12]:
inWidth = 452
inHeight = 452
thr = 0.2

BODY_PARTS = { "Nose": 0, "Neck": 1, "Right Shoulder": 2, "Right Elbow": 3, "Right Wrist": 4,
               "Left Shoulder": 5, "Left Elbow": 6, "Left Wrist": 7, "Right Hip": 8, "Right Knee": 9,
               "Right Ankle": 10, "Left Hip": 11, "Left Knee": 12, "Left Ankle": 13, "Right Eye": 14,
               "Left Eye": 15, "Right Ear": 16, "Left Ear": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "Right Shoulder"], ["Neck", "Left Shoulder"], ["Right Shoulder", "Right Elbow"],
               ["Right Elbow", "Right Wrist"], ["Left Shoulder", "Left Elbow"], ["Left Elbow", "Left Wrist"],
               ["Neck", "Right Hip"], ["Right Hip", "Right Knee"], ["Right Knee", "Right Ankle"], ["Neck", "Left Hip"],
               ["Left Hip", "Left Knee"], ["Left Knee", "Left Ankle"], ["Neck", "Nose"], ["Nose", "Right Eye"],
               ["Right Eye", "Right Ear"], ["Nose", "Left Eye"], ["Left Eye", "Left Ear"] ]

def pose_estimation(frame):
    quad={}
    frameWidth=frame.shape[1]
    frameHeight=frame.shape[0]
    net.setInput(cv.dnn.blobFromImage(frame,1.0,(inWidth,inHeight),(127.5,127.5,127.5),swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = [] # Key Points
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > thr else None)
    #print(points)
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            quad.__setitem__(partFrom,points[idFrom])
            quad.__setitem__(partTo,points[idTo])
    return points,quad

In [16]:
count=1
fields = ['Joint Name','Coordinates']
for file in path:
    img = cv.imread(file)
    points,cord=pose_estimation(img)
    """
    for key, val in cord.items():
        print()
    """
    print(points)
    print(cord)
    count+=1

[(1098, 303), (1098, 568), (909, 568), (795, 795), (909, 985), (1250, 568), (1440, 757), (1288, 985), (985, 1061), (871, 1477), (909, 1781), (1174, 1061), (1174, 1477), (1136, 1856), (1098, 265), (1136, 265), (1023, 303), (1174, 341), (2084, 0)]
[(1098, 303), (1098, 568), (909, 568), (795, 795), (909, 985), (1250, 568), (1440, 757), (1288, 985), (985, 1061), (871, 1477), (909, 1781), (1174, 1061), (1174, 1477), (1136, 1856), (1098, 265), (1136, 265), (1023, 303), (1174, 341), (2084, 0)]
{'Neck': (1098, 568), 'Right Shoulder': (909, 568), 'Left Shoulder': (1250, 568), 'Right Elbow': (795, 795), 'Right Wrist': (909, 985), 'Left Elbow': (1440, 757), 'Left Wrist': (1288, 985), 'Right Hip': (985, 1061), 'Right Knee': (871, 1477), 'Right Ankle': (909, 1781), 'Left Hip': (1174, 1061), 'Left Knee': (1174, 1477), 'Left Ankle': (1136, 1856), 'Nose': (1098, 303), 'Right Eye': (1098, 265), 'Right Ear': (1023, 303), 'Left Eye': (1136, 265), 'Left Ear': (1174, 341)}
[(308, 77), (279, 144), (241, 144